In [1]:
%env OPENAI_API_KEY = api_key

env: OPENAI_API_KEY=api_key


In [2]:
import glob
import json
import os
import re
import random

In [3]:
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
cd

/Users/swaran


In [5]:
cd desktop

/Users/swaran/Desktop


In [6]:
cd alljavajsons

/Users/swaran/Desktop/alljavajsons


In [7]:
inputFiles = []
correctOutputFiles = []
    
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

for file in sorted(glob.glob("*.java.json"), key=numericalSort):
    inputFiles.append(file)

for file in sorted(glob.glob("*.benchmark_log.json"), key=numericalSort):
    correctOutputFiles.append(file)

print(inputFiles)
print('\n\n\n')
print(correctOutputFiles)

['Android01.java.json', 'Android02.java.json', 'Android03.java.json', 'Android04.java.json', 'Android05.java.json', 'Android06.java.json', 'Android07.java.json', 'Android08.java.json', 'Android09.java.json', 'Android10.java.json', 'Android11.java.json', 'Android12.java.json', 'Android13.java.json', 'Android14.java.json', 'Android15.java.json', 'Android16.java.json', 'Android17.java.json', 'Android18.java.json', 'Android19.java.json', 'Android20.java.json', 'Android21.java.json', 'Android22.java.json', 'Android23.java.json', 'Android24.java.json', 'Android25.java.json', 'Android26.java.json', 'Android27.java.json', 'Android28.java.json', 'Android29.java.json', 'Android30.java.json', 'Android31.java.json', 'Android32.java.json', 'Android33.java.json', 'Android34.java.json', 'Android35.java.json', 'Android36.java.json', 'Android37.java.json', 'Android38.java.json', 'Android39.java.json', 'Android40.java.json', 'Android41.java.json', 'Android42.java.json', 'Android43.java.json', 'Android44

In [8]:
codes = []

for code in inputFiles:
    codes.append(str(json.load(open(code))['originalContent']))

print(codes)

['package androidExamples;\n\npublic class Android01 extends Activity {\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n\n        mTextView = new TextView(this);\n\n        if (savedInstanceState == null) {\n            mTextView.setText("Welcome to HelloAndroid!");\n        } else {\n            mTextView.setText("Welcome back.");\n        }\n\n        setContentView(mTextView);\n    }\n\n    private TextView mTextView = null;\n}', 'package androidExamples;\n\n\n\npublic class Android02 extends Activity\n{\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n        setContentView(R.layout.main);\n    }\n}', 'package androidExamples;\n//ID = 26362\n\npublic class Android03 extends ItemizedOverlay<OverlayItem>{\n\n\tpublic Android03(Drawable default

In [9]:
correct_outputs = []
for output in correctOutputFiles:
    correct_output_list = json.load(open(output))['correct_imports']
    correct_output_list = ["import "+i+";" for i in correct_output_list]
    correct_outputs.append(correct_output_list)

print(correct_outputs)

[['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.os.Bundle;', 'import android.app.Activity;'], ['import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.ItemizedOverlay;', 'import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.widget.TabHost;', 'import android.widget.TabHost.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.widget.LinearLayout;', 'import android.view.ViewGroup;', 'import android.view.Gravity;'], ['import android.widget.ArrayAdapter;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.app.Activity;'], ['import java.net.InetAddress;', 'import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Lo

In [10]:
print(correct_outputs[0])

['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;']


In [11]:
dataset = {
    "codes": codes,
    "correct_outputs": correct_outputs
}

In [12]:
# print(dataset)

In [13]:
sample_list = []

for i in range(0, 10):
    sample_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))

print(sample_list)

[{'codes': 'package androidExamples;\n\npublic class Android01 extends Activity {\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n\n        mTextView = new TextView(this);\n\n        if (savedInstanceState == null) {\n            mTextView.setText("Welcome to HelloAndroid!");\n        } else {\n            mTextView.setText("Welcome back.");\n        }\n\n        setContentView(mTextView);\n    }\n\n    private TextView mTextView = null;\n}', 'correct_outputs': ['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;']}, {'codes': 'package androidExamples;\n\n\n\npublic class Android02 extends Activity\n{\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n        setContentView(R.layout.main);\n    }\n}', 'cor

In [14]:
test_examples = [(example["codes"], example["correct_outputs"]) for example in sample_list]
y_true = [correct_outputs for _, correct_outputs in test_examples]

In [15]:
code_snippet = test_examples[0][0]

# Zero-Shot, One-Shot, and Few-Shot Learning
### Zero-Shot implementation

In [16]:
def get_prediction(engine, code_snippet):
    response = openai.Completion.create(
        engine=engine,
        prompt=f"Only give correct imports:\n\"\"\"{code_snippet}\"\"\"",
        temperature=0,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0
    )
    
    return response
get_prediction("text-davinci-003", code_snippet)

<OpenAIObject text_completion id=cmpl-6bgEZZcnq1JlPwIhBrO8oLpLol5hx at 0x1057ea860> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nimport android.app.Activity;\nimport android.os.Bundle;\nimport android.widget.TextView;"
    }
  ],
  "created": 1674438503,
  "id": "cmpl-6bgEZZcnq1JlPwIhBrO8oLpLol5hx",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 27,
    "prompt_tokens": 149,
    "total_tokens": 176
  }
}

In [17]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


def get_predictions(engine, dataset):
    y_pred = []
    for code_snippet, correct_imports in tqdm(dataset):
        y_pred.append(get_prediction(engine, code_snippet)['choices'][0]['text'].strip().split("\n"))
    return y_pred

def pred_process(y_pred, y_true):
    y_pred_processed = []
    y_true_processed = []
    
    for pred, correct_imports in zip(y_pred, y_true):
        max_length = max(len(pred), len(correct_imports))
        correct_preds = list(set(pred).intersection(correct_imports))
#         print('Correct Predictions:', correct_preds)
#         wrong_preds = max_length - len(correct_preds)
#         print('Wrong Predictions:', wrong_preds)

        for i in range(0, max_length):
            if i<len(correct_preds):
                y_pred_processed.append(1)
                y_true_processed.append(1)
            else:
                if i<len(correct_imports):
                    y_pred_processed.append(0)
                    y_true_processed.append(1)
                else:
                    y_pred_processed.append(1)
                    y_true_processed.append(0)
            
    print(y_pred_processed)
    print(y_true_processed)
    return y_pred_processed, y_true_processed

def eval_performance(y_pred, y_true):
    print(json.dumps({
        "accuracy": accuracy_score(y_pred, y_true),
        "f1": f1_score(y_pred, y_true),
        "precision": precision_score(y_pred, y_true),
        "recall": recall_score(y_pred, y_true)
    }, indent=2))

In [18]:
y_pred = get_predictions("text-davinci-003", test_examples)

100%|███████████████████████████████████████████| 10/10 [00:15<00:00,  1.50s/it]


In [19]:
print(y_true)

[['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.os.Bundle;', 'import android.app.Activity;'], ['import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.ItemizedOverlay;', 'import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.widget.TabHost;', 'import android.widget.TabHost.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.widget.LinearLayout;', 'import android.view.ViewGroup;', 'import android.view.Gravity;'], ['import android.widget.ArrayAdapter;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.app.Activity;'], ['import java.net.InetAddress;', 'import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Lo

In [20]:
print(y_pred)

[['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;', 'import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import android.widget.TabHost;', 'import android.widget.TabSpec;'], ['import android.view.Gravity;', 'import android.view.ViewGroup;', 'import android.widget.LinearLayout;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.widget.ArrayAdapter;'], ['import java.net.Inet4Address;', 'import java.net.InetAddress;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Log;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'im

In [21]:
y_pred_processed, y_true_processed = pred_process(y_pred, y_true)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [22]:
eval_performance(y_pred_processed, y_true_processed)

{
  "accuracy": 0.9142857142857143,
  "f1": 0.955223880597015,
  "precision": 0.9142857142857143,
  "recall": 1.0
}


### One-Shot and Few-Shot implementation

In [23]:
def get_fewshot_prediction(engine, examples, code):
    context_code = ""
    for code_snippet, correct_output in examples:
        output = '\n'.join(correct_output)
        context_code += f"Code:\n\"\"\"{code_snippet}\"\"\"\nOutput:\n{output}\n"
    
    prompt=f"Only give correct imports:\n\"\"\"{context_code}\"\"\"Code:\"\"\"{code}\"\"\"\nOutput: "
    response = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        temperature=0,
        max_tokens=1000,
        top_p=1,
        frequency_penalty=0.2,
        presence_penalty=0
    )
    return response

In [24]:
def get_context_prediction(engine, context, dataset):
    y_pred = []
    for code, import_lines in tqdm(dataset):
        y_pred.append(get_fewshot_prediction(engine, context, code)['choices'][0]['text'].strip().split("\n"))
    return y_pred

In [25]:
one_or_fewshot_list = []

for i in range(0, 10):
    one_or_fewshot_list.append(dict(codes=dataset["codes"][i], correct_outputs=dataset["correct_outputs"][i]))

one_or_few_example = [(example["codes"], example["correct_outputs"]) for example in one_or_fewshot_list]
one_or_few_sample = random.sample(one_or_few_example, 5)

print(one_or_few_sample)

[('package androidExamples;\n//ID = 526524\npublic class Android10 {\n\n\tpublic static void main(String[] args) {\n\t\t\n\t\t// TODO Auto-generated method stub\n\t\tChronometer t = (Chronometer)findViewById(R.id.toptime);\n\t\tlong time = SystemClock.elapsedRealtime()-t.getBase();\n\t\tLog.d(null,"Was: "+time); //time is not the proper time for some reason - it is a random number between 0 and 50\n\t\tt.setBase(SystemClock.elapsedRealtime());\n\t\tt.start();\n\n\t}\n\n\tprivate static Chronometer findViewById(int toptime) {\n\t\t// TODO Auto-generated method stub\n\t\treturn null;\n\t}\n\n}', ['import android.widget.Chronometer;', 'import android.os.SystemClock;', 'import android.util.Log;']), ('package androidExamples;\n\npublic class Android01 extends Activity {\n    /** Called when the activity is first created. */\n    @Override\n    public void onCreate(Bundle savedInstanceState) {\n        super.onCreate(savedInstanceState);\n\n        mTextView = new TextView(this);\n\n        

### Only one sample used for One-Shot

In [26]:
y_pred = get_context_prediction("text-davinci-003", [one_or_few_sample[0]], test_examples)

100%|███████████████████████████████████████████| 10/10 [00:18<00:00,  1.82s/it]


In [27]:
print(y_true)

[['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.os.Bundle;', 'import android.app.Activity;'], ['import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.ItemizedOverlay;', 'import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.widget.TabHost;', 'import android.widget.TabHost.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.widget.LinearLayout;', 'import android.view.ViewGroup;', 'import android.view.Gravity;'], ['import android.widget.ArrayAdapter;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.app.Activity;'], ['import java.net.InetAddress;', 'import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Lo

In [28]:
print(y_pred)

[['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.GeoPoint;', 'import android.graphics.drawable.Drawable;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.widget.TabHost;', 'import android.widget.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.view.ViewGroup;', 'import android.widget.LinearLayout;', 'import android.view.Gravity;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.ArrayAdapter;', 'import android.view.View;'], ['import java.net.InetAddress;', 'import java.net.Inet4Address;', 'import java.net.UnknownHostException;', 'import java.io.IOException;', 'import java.net.Socket;'], ['import android.os.Bundle;', 

In [29]:
y_pred_processed, y_true_processed = pred_process(y_pred, y_true)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [30]:
eval_performance(y_pred_processed, y_true_processed)

{
  "accuracy": 0.9428571428571428,
  "f1": 0.9705882352941176,
  "precision": 0.9428571428571428,
  "recall": 1.0
}


### Few sample used randomly for Few-Shot

In [31]:
random.shuffle(one_or_few_sample)
context = one_or_few_sample[:3]
random.shuffle(context)

In [32]:
y_pred = get_context_prediction("text-davinci-003", context, test_examples)

100%|███████████████████████████████████████████| 10/10 [00:14<00:00,  1.49s/it]


In [33]:
print(y_true)

[['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.os.Bundle;', 'import android.app.Activity;'], ['import com.google.android.maps.OverlayItem;', 'import com.google.android.maps.ItemizedOverlay;', 'import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;'], ['import android.os.Bundle;', 'import android.widget.TextView;', 'import android.app.Activity;'], ['import android.widget.TabHost;', 'import android.widget.TabHost.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.widget.LinearLayout;', 'import android.view.ViewGroup;', 'import android.view.Gravity;'], ['import android.widget.ArrayAdapter;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.app.Activity;'], ['import java.net.InetAddress;', 'import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Lo

In [34]:
print(y_pred)

[['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.app.Activity;', 'import android.os.Bundle;'], ['import android.graphics.drawable.Drawable;', 'import com.google.android.maps.GeoPoint;', 'import com.google.android.maps.ItemizedOverlay;', 'import com.google.android.maps.OverlayItem;'], ['import android.app.Activity;', 'import android.os.Bundle;', 'import android.widget.TextView;'], ['import android.widget.TabHost;', 'import android.widget.TabSpec;', 'import android.graphics.drawable.Drawable;'], ['import android.view.ViewGroup;', 'import android.view.Gravity;', 'import android.widget.LinearLayout;'], ['import android.widget.ArrayAdapter;', 'import android.os.Bundle;', 'import android.view.View;', 'import android.app.Activity;'], ['import java.net.InetAddress;', 'import java.io.IOException;', 'import java.net.Inet4Address;', 'import java.net.Socket;', 'import java.net.UnknownHostException;', 'import android.util.Log;'], ['

In [35]:
y_pred_processed, y_true_processed = pred_process(y_pred, y_true)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [36]:
eval_performance(y_pred_processed, y_true_processed)

{
  "accuracy": 0.9714285714285714,
  "f1": 0.9855072463768115,
  "precision": 0.9714285714285714,
  "recall": 1.0
}
